Notebook prepared by **Dr James Trayford** - for queries please email [`james.trayford@port.ac.uk`](mailto:james.trayford@port.ac.uk)

To use this notebook (if you haven't already) you can first save a copy to your local drive by clicking `File > Save a Copy in Drive` and run on that copy. `Edit > Clear all outputs` on that copy should also ensure yopu have a clean version
 to start from.

# **0. Introduction**

We will be using the [STRAUSS code](https://github.com/james-trayford/strauss) for this activity

<img src="https://github.com/james-trayford/strauss/blob/main/misc/strauss_logo.png?raw=true">

For reference, you can read an overview of the code (as well as detailed documentation) [at this link](https://strauss.readthedocs.io/en/latest/)


`strauss` is an open source, object-oriented python library intended to be a flexible toolkit and engine for sonification, allowing detailed low-level control over the sonification process. Simultaneously, casual users can quickly hear their data, adapting a library of python notebook templates for a range of applications.

By analogy to visualisation, the intention is to provide something akin to a plotting library. A library allows users to make a variety of simple plots easily, but also the option to control all aspects of plots and adapt them to the intricacies of their data, for optimal presentation.

By adopting a general approach, strauss is intended to sonify any form of data for users with differing expertise. strauss is work in progress, and benefits form user feedback - filling in this feedback will be very useful in making the code better!

# 0.1 This notebook:

This notebook will demonstrate some of the ways in which `strauss` can be applied to sonify solar data. Alternative options may be demonstrated with commented out code ( i.e. lines of actual code preceded by `#`) - feel free to try these! Generally the goal of this notebook is to give some open examples to explore the code and experiment, so please do so! 

***Throughout, we will use the pencil emoji, "✏️", to indicate parameters you might want to experiment with...***

# 1. Set-Up

First, let's install `strauss`! Just run the code cell below.

*We will use the `spectraliser` development branch for this notebook to play with some experimental features.* 

Install can take a while - but you should only need to run it once!*

In [ ]:
 !pip --quiet install git+https://github.com/james-trayford/strauss.git@spectraliser -U

and also make a local copy of the repository

In [ ]:
 !git clone https://github.com/james-trayford/strauss.git

Make plots appear in-line by default, import the modules we need, and set some figure defaults...


In [ ]:
%matplotlib inline

from scipy.io import readsav 
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import urllib.request
import os
import zipfile
import glob

from scipy.signal import savgol_filter as sgf
matplotlib.rcParams.update({'font.size': 16, 'figure.figsize': (12,6)})

# strauss imports
from strauss.sonification import Sonification
from strauss.sources import Events, Objects
from strauss import channels
from strauss.score import Score
from strauss.generator import Sampler, Synthesizer, Spectralizer
from strauss import sources as Sources 
import strauss

# modules to display in-notebook
import IPython.display as ipd
from IPython.core.display import display, Markdown, Latex, Image

# 2. Getting the data and fitting functions

We'll be handling line-fit data today which is relative light and easy to process in this environment - though most of these ideas apply to the raw data set too.

First, let's run a script to download the data:

In [ ]:
outdir = "./solar_data"

path = os.path.realpath(outdir)
if not glob.glob(outdir): 
  os.mkdir(path)  
    
fname = "WISA2023_STRAUSS_tutorial"
url = "https://drive.google.com/uc?export=download&id=1xYb2ewmuB1ABlAcp0bwWCs89u__-qJ8g"
print(f"Downloading files...")
with urllib.request.urlopen(url) as response, open(f"{path}/{fname}", 'wb') as out_file:
  data = response.read() # a `bytes` object
  out_file.write(data)

print(f"Unzipping files to {outdir}/solar_data ...")
with zipfile.ZipFile(f"{outdir}/{fname}", 'r') as zip_ref:
    zip_ref.extractall(f"{outdir}")

print(f"Clearing up...")
os.remove(f"{path}/{fname}")

print("Done.")

Now lets read this into a data dictionary ("`dict`") object...

In [ ]:
lines = ['MgII','SiIV']
wlens = [2796, 1403]

data = {}

for l in lines:
    f = readsav(f'data/maxval_2014-04-03{l}.sav')
    
    data[l] = {}

    # clean up NaNs
    mdopr_valid = ~np.isnan(f['mdopr'])
    mdopb_valid = ~np.isnan(f['mdopb'])
    mnth_valid = ~np.isnan(f['mnth'])
    mint_valid = ~np.isnan(f['mint'])

    # order data in time
    tdopr_sort = np.argsort(f['tdopr'][mdopr_valid])
    tdopb_sort = np.argsort(f['tdopb'][mdopb_valid])
    tnth_sort = np.argsort(f['tnth'][mnth_valid])
    tint_sort = np.argsort(f['tint'][mint_valid])

    # Red shift velocities for Mg II [km/s]
    data[l]['mdopr'] = f['mdopr'][mdopr_valid][tdopr_sort]
    # Blue shift velocities for Mg II [km/s]
    data[l]['mdopb'] = f['mdopb'][mdopb_valid][tdopb_sort]
    
    # Time positions for red shift velocities for Mg II [min]
    data[l]['tdopr'] = f['tdopr'][mdopr_valid][tdopr_sort]
    #  Time positions for blue shift velocities for Mg II [min]
    data[l]['tdopb'] = f['tdopb'][mdopb_valid][tdopb_sort]
    # Standard deviations for red shift velocities for Mg II [km/s]
    data[l]['stddopr'] = f['stddopr'][mdopr_valid][tdopr_sort]
    # Standard deviations for blue shift velocities for Mg II [km/s]
    data[l]['stddopb'] = f['stddopb'][mdopb_valid][tdopb_sort] 
    # Non-thermal velocities for Mg II [km/s]
    data[l]['mnth'] = f['mnth'][mnth_valid][tnth_sort]
    # Times for the non-thermal velocities for Mg II [min]
    data[l]['tnth'] = f['tnth'][mnth_valid][tnth_sort]
    # Standard deviation for the non-thermal velocities for Mg II [km/s]
    data[l]['stdnth'] = f['stdnth'][mnth_valid][tnth_sort]
    # Integrated intensities for Mg II [DN]
    data[l]['mint'] = f['mint'][mint_valid][tint_sort]
    # Times for the integrated intensities for Mg II [min]
    data[l]['tint'] = f['tint'][mint_valid][tint_sort]
    
    data[l]['vgridb'] = np.column_stack([np.linspace(-250,0,2000)]*data[l]['mdopb'].size)
    data[l]['vgridr'] = np.column_stack([np.linspace(0,250,2000)]*data[l]['mdopr'].size)

Let's now define a Gaussian function, so we can reproduce the fitted line profiles...

In [ ]:
def gaussian(x, mu, sig):
    # Gaussian function to reproduce fitted line profiles 
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))/(sig * np.sqrt(2*np.pi))

...and look at the Doppler velocity data for one line...

In [ ]:
# choose a line 0 => MgII, 1 => SiIV
l = lines[0]

# show doppler velocity evolution
plt.title(l)
plt.scatter(data[l]['tdopb'], 
            data[l]['mdopb'])
plt.scatter(data[l]['tdopr'], 
            data[l]['mdopr'])
plt.xlabel('Time [min]')
plt.ylabel('Doppler Velocity [km/s]')

Finally, we might also want to smooth the data to reduce noisy, short-time variations in trends when sonifying in certain contexts. This also allows us to interpolate the data, such that it is all on the same time grid. 

We smooth it on a chosen timescale, "`tsmooth`" (5 minutes by default).

In [ ]:
# fine time grid for smoothing data
tfine = np.linspace(0,96.5,10000)

# dict for smoothed data
sdata = {}

# pick a smoothing timescale
tsmooth = 5 # minutes

for l in lines:
    winlen = int(tsmooth/np.diff(tfine)[0])
    winlen += 1-(winlen % 2)
    sdata[l] = {}
    
    # smooth the data arrays onto a regular fine time grid
    sdata[l]['mdopb'] = sgf(np.interp(tfine, data[l]['tdopb'], data[l]['mdopb']), winlen, 3)
    sdata[l]['stddopb'] = sgf(np.interp(tfine, data[l]['tdopb'], data[l]['stddopb']), winlen, 3)
    sdata[l]['mdopr'] = sgf(np.interp(tfine, data[l]['tdopr'], data[l]['mdopr']), winlen, 3)
    sdata[l]['stddopr'] = sgf(np.interp(tfine, data[l]['tdopr'], data[l]['stddopr']), winlen, 3)
    sdata[l]['mnth'] = sgf(np.interp(tfine, data[l]['tnth'], data[l]['mnth']), winlen, 3)
    sdata[l]['stdnth'] = sgf(np.interp(tfine, data[l]['tnth'], data[l]['stdnth']), winlen, 3)
    sdata[l]['mint'] = sgf(np.interp(tfine, data[l]['tint'], data[l]['mint']), winlen, 3)
    
    sdata[l]['vgridb'] = np.column_stack([np.linspace(-250,0,2000)]*tfine.size)
    sdata[l]['vgridr'] = np.column_stack([np.linspace(0,250,2000)]*tfine.size)

Finally, lets plot all of that together...

In [ ]:
# choose a line and blue ('b') or red ('r') shifted component to plot
l = lines[0]
c = 'b'

# plot some properties
print(f"{l}: {c.upper()}-shifted component")
fig = plt.figure(figsize=(10,10))
fig.add_subplot(211)
plt.scatter(data[l][f'tdop{c}'],data[l][f'mdop{c}'],s=5, label='Raw Frames')
plt.plot(tfine,sdata[l][f'mdop{c}'], zorder=9,lw=8, c='w')
plt.plot(tfine,sdata[l][f'mdop{c}'], zorder=10,lw=2,c='0.3', label ="Moving Average")
plt.legend(frameon=0)
plt.ylabel('Doppler Velocity [km/s]')
fig.add_subplot(212)
plt.scatter(data[l][f'tdop{c}'],data[l][f'stddop{c}'],s=5)
plt.plot(tfine,sdata[l][f'stddop{c}'], zorder=9,lw=8, c='w')
plt.plot(tfine,sdata[l][f'stddop{c}'], zorder=10,lw=2,c='0.3',)
plt.ylabel('Standard Deviation [km/s]')
plt.xlabel('Time [min]')
print(sdata[l][f'stddop{c}'])

# 3. Sonification: Abstracted approaches

In this section, we use the line fitting parameters mapped to different properties of sound, to convey their evolutino over time

## 3.1 Listening to the Raw, discrete, 1D Data Series using `Events`

Here we will sonify solar line-fit data as a ***one-dimensional time series*** , where some ***sound property*** is varied with ***time*** in the ***sonification***, in the same way that **doppler velocity**, varies with ***time*** in the ***evolving line-profile fits*** (early in the sonification is earlier times, later is later times). This approach in general is covered in this [activity workbook](https://github.com/james-trayford/AudibleUniverseWorkbooks/blob/group4/STRAUSSdemo.ipynb), for the 2022 _'Audible Universe'_ meeting.

In `strauss` we could treat each ***doppler velocity*** data point in the spectra as separate audio `Events`, with an occurence `time` mapped from their ***observation time***. 

However, articulating each data point as a separate ***note*** for ***many thousands*** of data points can require long and drawn-out sonifications. 

Here we demonstrate this approach with just a portion of the data points (staying within `Colab`'s RAM limitations for unpaid users 🤫). We use the `Synthesizer` object with the `pitch_mapper` preset by default - this has a default pitch range of ***two octaves*** (a factor of 4 in frequency) and we pick an `E3` note (165 Hz) as the base (lowest) frequency.

We will hear the **doppler velocity** at each instant as a `pitch`, with their observation time mapped to the occurence `time` in the sonification (moving from early to late). We can hear the noisy beahaviour, but also general trends, particular the rising blue-shifts at the end. 

### 3.1.1 Atonal Example 

Allowing the pitch to freely vary with doppler velocity, we here an exact mapping between relative frequency and the doppler shift

In [ ]:
display(Markdown(f"### Sonifying in 1D using '`Events`' object:"))

# Pick a line... ✏️
l = 'MgII'

# show spectrum again, for reference
plt.scatter(data[l]['tdopb'], -data[l]['mdopb'], s=4)
plt.ylabel('Doppler Blue-Shift Velocity [km/s]')
plt.xlabel('Time [m]')
plt.show()

%matplotlib notebook
# specify the base notes used. In this example we use a single E3 note and 
# freely vary the pitch via the 'pitch_shift' parameter 
notes = [["E3"]]

# we could also just specify a particular frequency... ✏️
# notes = [[150.]]

score =  Score(notes, 30)


maps = {'pitch':np.ones(data[l]['mdopb'].size),
        'time': data[l]['tdopb'],
        'pitch_shift':-data[l]['mdopb']}

# specify audio system (e.g. mono, stereo, 5.1, ...)
system = "mono"

# set up synth (this generates the sound using mathematical waveforms)
generator = Synthesizer()
generator.load_preset('pitch_mapper')

# or maybe the sampler instead by uncommenting this block (this uses recorded audio clips)
# generator = Sampler(sampfiles="./strauss/data/samples/mallets")
# generator.modify_preset({'phi': 0,'theta':0,})

generator.modify_preset({'note_length':0.1,
                         'volume_envelope': {'use':'on',
                                             # A,D,R values in seconds, S sustain fraction from 0-1 that note 
                                             # will 'decay' to (after time A+D)
                                             'A':0.02,    # ✏️ for such a fast sequence, using ~10 ms values 
                                             'D':0.04,    # ✏️ for such a fast sequence, using ~10 ms values
                                             'S':0.,      # ✏️ decay to volume 0
                                             'R':0.001}}) # ✏️ for such a fast sequence, using ~10 ms values

# set 0 to 100 percentile limits so the full pitch range is used...
# setting 0 to 101 for pitch means the sonification is 1% longer than the final note position
lims = {'time': ('0','100.333333'),
        'pitch_shift': ('0','100')}

# set up source
sources = Events(maps.keys())
sources.fromdict(maps)
sources.apply_mapping_functions(map_lims=lims)

soni = Sonification(score, sources, generator, system)
soni.render()
# soni.save('pitchpm.wav')
dobj = soni.notebook_display()
%matplotlib inline

### 3.1.2 "Musical" Example

Instead, in this example restrict notes onto a (western) musical scale - the major pentatonic. Pitch is again mapped to doppler velocity, but now binned onto a discrete set of musical pitches. In some contexts, this may be preferable.

In [ ]:
display(Markdown(f"### Sonifying in 1D using '`Events`' object:"))

# Pick a line...
l = 'MgII'Allowing the pitch to freely vary with doppler velocity, we here an exact mapping between relative frequency and the doppler shift

# grab times and values for chosen line
x = data[l]['tdopb']
y = -data[l]['mdopb']

# show light curve again, for reference
plt.scatter(time, flux, s=4)
plt.ylabel('Doppler Velocity [km/s]')
plt.xlabel('Time [min]')
plt.show()

%matplotlib notebook

# specify the notes used. 

# C major pentatonic
notes = [["C3","E3","F3","G3","B3","C4","E4","F4","G4","B4","C5","E5","F5","G5","B5"]]

# or a whole-tone scale, (was it just a dream... a dream... a dream...)
#
# This is a symetrical scale and here also demonstrates using raw frequencies in Hz.
# The whole tone scale uses 2 semitone jumps, what about minor thirds (3) fourths (5)
# or fifths (7)? Specify `semitones` to change this.
# `nint` specifies the number of intervals in the scale. Note a bigger semitone interval
# will reach higher pitches (perhaps inaudible ones...) ✏️

# semitones = 2.
# nint = 10
# notes = [100*2**(np.arange(nint)*(semitones/12))]

# ------
# In this context, we also consider the `Score` optional parameter `pitch_binning`.
#
# This determines how we bin the 'pitch' quantity, which can be either:
# - 'uniform':  the range of mapped values for 'pitch' are split into even bins
#               representing  each of the scored notes
# - 'adaptive': the range of mapped values for 'pitch' split by percentile. 
#               such that each interval is played approximately the same number of 
#               times
# without specifying,  'adaptive' is used by default, but we specify 'uniform' here
#
# which is better for representing this data?

# we specify 'uniform' pitch binning in the primary example... ✏️
score =  Score(notes, 30, pitch_binning='uniform')

# what about adaptive?
#score =  Score(notes, 30, pitch_binning='adaptive')

maps = {'pitch':y,
        'time': x}

# specify audio system (e.g. mono, stereo, 5.1, ...)
system = "mono"

# set up synth (this generates the sound using mathematical waveforms)
generator = Synthesizer()
generator.load_preset('pitch_mapper')

generator.modify_preset({'note_length':0.15,
                         'volume_envelope': {'use':'on',
                                             # A,D,R values in seconds, S sustain fraction from 0-1 that note 
                                             # will 'decay' to (after time A+D)
                                             'A':0.02,    # ✏️ for such a fast sequence, using ~10 ms values 
                                             'D':0.04,    # ✏️ for such a fast sequence, using ~10 ms values
                                             'S':0.,      # ✏️ decay to volume 0
                                             'R':0.001}}) # ✏️ for such a fast sequence, using ~10 ms values

# set 0 to 100 percentile limits so the full pitch range is used...
# setting 0 to 101 for pitch means the sonification is 1% longer than 
# the time needed to trigger each note - by making this more than 100%
# we give all the notes time to ring out (setting this at 100% means
# the final note is triggered at the momement the sonification ends)
lims = {'time': ('0','101'),
        'pitch_shift': ('0','100'),
        'pitch': ('0','100')}

# set up source
sources = Events(maps.keys())
sources.fromdict(maps)
sources.apply_mapping_functions(map_lims=lims)

soni = Sonification(score, sources, generator, system)
soni.render()
dobj = soni.notebook_display()
%matplotlib inline

## 3.2 Listening to the smoothed line profile evolution using `Objects` 

Now, we use the smoothed properties we calculated to map properties continuosly using the `Object` source type. 

We are considering the blue-shifted component of the `MgII` and `SiIV` lines together in a single sonification. Using stereo separation, we place the `MgII` `Object` source all the way left and `SiIV` `Object` source all the way right - _definitely helps to have headphones!_

here we start the examples at the same base `pitch` and map doppler velocities to `pitch_shift`, so their divergence from being in tune tells us how strong the relative doppler shifts are.

Optionally, we can also make each `Object` source multivariate by also encoding the doppler velocity ***standard deviation***  to another property of sound. Here we use the intensity of a volume low-frequency oscillator (LFO). This modulates the volume with a fast rhythmic pulse (12 Hz bty default), suh that  the pulse intensity increases with the standard deviation. In this way, we can hear the velocity and it's standard deviation simultaneously

To turn this on, reference the "✏️" symbols below... 

In [ ]:
# specify audio system (e.g. mono, stereo, 5.1, ...)
system = "stereo"

# length of the sonification in s ✏️
length = 30.

# set up synth and turn on LP filter
generator = Synthesizer()
generator.load_preset('pitch_mapper')
generator.preset_details('pitch_mapper')

doppler_mapping = 'pitch_shift' # also try 'cutoff'

display(Markdown(f"### Sonifying in 1D using '`pitch_shift`':"))

notes = [["A2", "A2"]]
score =  Score(notes, length)

# "True" makes this a multivariate sonification where intensity of pulse is mapped to standard deviation... ✏️
use_lfo =  False 

lfo_type = ['pitch', 'volume']
lfo_idx = 1

generator.modify_preset({'filter':'on',
                                        f"{lfo_type[lfo_idx]}_lfo": {"use": use_lfo, 
                                        "amount":0, 
                                        "freq":12, #  base LFO frequency in Hz (12) ✏️
                                        "phase":0}})

maps = {'pitch':[0,1],
        'time_evo':[tfine, tfine],
        doppler_mapping : [(-sdata['MgII'][f'mdopb']), (-sdata['SiIV'][f'mdopb'])],
        f'{lfo_type[lfo_idx]}_lfo/amount':[sdata['MgII'][f'stddopb'],sdata['SiIV'][f'stddopb']],
        'phi':[0.25,0.75],
       'theta':[0.5,0.5]}

# set 0 to 100 percentile limits so the full pitch and time range is used...
lims = {'time_evo': ('0','100'),
        doppler_mapping: ('0','100'),
        f'{lfo_type[lfo_idx]}_lfo/amount': ('0','100')}

# set up source
sources = Objects(maps.keys())
sources.fromdict(maps)
sources.apply_mapping_functions(map_lims=lims)

soni = Sonification(score, sources, generator, system)
soni.render()
dobj = soni.notebook_display()
%matplotlib inline

We can play around with some of these `evolvable` properties (listed below) here. Are all of these effective for this data? Could they be effective for other types of data representations?

In [ ]:
display(Markdown(f"### ***'Mappable'*** properties:"))
for m in Sources.mappable:
  display(Markdown(f' * `{m}` '))

display(Markdown(f"### ***'Evolvable'*** properties:"))
for m in Sources.evolvable:
  display(Markdown(f' * `{m}` '))

In [ ]:
# specify audio system (e.g. mono, stereo, 5.1, ...)
system = "stereo"

# length of the sonification in s ✏️
length = 30.

# set up synth and turn on LP filter
generator = Synthesizer()
generator.load_preset('default')
generator.preset_details('default')

doppler_mapping = 'cutoff' # also try 'cutoff'

display(Markdown(f"### Sonifying in 1D using '`pitch_shift`':"))

notes = [["A2", "E3"]]
score =  Score(notes, length)

# "True" makes this a multivariate sonification where intensity of pulse is mapped to standard deviation... ✏️
use_lfo =  True 

lfo_type = ['pitch', 'volume']
lfo_idx = 1

generator.modify_preset({'filter':'on',
                        f"{lfo_type[lfo_idx]}_lfo": {"use": use_lfo, 
                                                     "amount":0, 
                                                     "freq":12, #  base LFO frequency in Hz (12) ✏️
                                                     "phase":0}})

maps = {'pitch':[0,1],
        'time_evo':[tfine, tfine],
        doppler_mapping : [(-sdata['MgII'][f'mdopb']), (-sdata['SiIV'][f'mdopb'])],
        f'{lfo_type[lfo_idx]}_lfo/amount':[sdata['MgII'][f'stddopb'],sdata['SiIV'][f'stddopb']],
        'phi':[0.25,0.75],
       'theta':[0.5,0.5]}

# set 0 to 100 percentile limits so the full pitch and time range is used...
lims = {'time_evo': ('0','100'),
        doppler_mapping: ('0','100'),
        f'{lfo_type[lfo_idx]}_lfo/amount': ('0','100')}

# set up source
sources = Objects(maps.keys())
sources.fromdict(maps)
sources.apply_mapping_functions(map_lims=lims, param_lims={'cutoff': (0.2, 0.8)})

soni = Sonification(score, sources, generator, system)
soni.render()
dobj = soni.notebook_display()
%matplotlib inline

## 3.X A Side Note About Presets

You can see all the parameters that make up the default `Synthesizer` preset in a pop-up window:

In [ ]:
%pycat ./strauss/src/strauss/presets/synth/default.yml

... as well as the other presets we might want to use

In [ ]:
ls -1 ./strauss/src/strauss/presets/synth/*.yml

We can modify these presets at runtime (as we do in various examples throughout this session), or even write our own presets in `.yml` format.

The `generator.preset()` function also accepts a filepath to a custom presets, where any changed preset parameters replace those in the `default` preset. 

# 4. Sonification: Spectral Representation

Haing covered some of these more abtracted approaches, let's consider a direct sonification of the spectrum or _"Spectralisation"_

## 4.1 Using an array of oscillators (slow)

Here, we convert the frequencies representing these two spectral lines directly to oscillators art audible frequencies, that increase involume to represent their flux density

First, we reconstruct the line spectrum representing the `SiIV` and `MgII` combination, using the fitting parameter data provided. These are mapped onto musical notes a _"perfect fifth"_ apart, such that these sound 'in tune' without any relative shift, but detuned as their blueshifts diverge. 

In this example we manipulate the data somewhat; scaling up the relative doppler shifts by a factor "`scale`" so that these frequency changes are audible to most people. We also apply a "`contrast`" to the intensities to control the relative power of each line (and ensure both are well audible)

Generally, this approach allows us to smoothly vary spectra in time, but is relatively slow compared to the IFFT technique that is under construction in STRAUSS (see §4.2)

In [ ]:
c = 3e5 # speed of light in km/s
npoints = 500 # number of frequency points per line

# subjective parameters
scale = 500 # factor to scale up frequency variation! ✏️
contrast = 0.5 # index to raise the intensties by. 0 < contrat <= 1 will ✏️
notes = [300, 600*4/3]  # representative notes in Hz for each line ✏️

wavs = []
fluxes = []

for i in range(len(wlens)):
    z = -sdata[lines[i]]['mdopb']*scale/c
    std = sdata[lines[i]]['stddopb']*scale/c
    wavpls = (z + 1).T*wlens[i] - 10*std
    wav = np.linspace(*np.percentile((z + 1).T*wlens[i] + np.expand_dims([-5*std, 5*std],1)*wlens[i], [0,100]),
                      npoints)
    wav *= notes[i]/wlens[i]
    wavs.append(wav)
    fluxes.append(gaussian(np.column_stack([wav]*z.size), (1+z)*notes[i], std*notes[i])*sdata[lines[i]]['mint']**contrast)
    
wavs = np.hstack(wavs)
fluxes = np.vstack(fluxes)

and sonify...

In [ ]:
# specify audio system (e.g. mono, stereo, 5.1, ...)
system = "stereo"

# length of the sonification in s
length = 30.

# set up synth and turn on LP filter
generator = Synthesizer()
generator.load_preset('spectraliser')
generator.preset_details('spectraliser')

display(Markdown(f"### Sonifying in 1D using '`pitch_shift`':"))

score =  Score([list(wavs)], length)

maps = {'pitch':list(range(wavs.size)),
        'time_evo':[tfine[:]]*wavs.size,
        'volume':list(fluxes[:,:]),
        'phi':list(0.25 + 0.5*wavs/wavs.max()),
        'theta':[0.5]*wavs.size}

# set 0 to 100 percentile limits so the full pitch and time range is used...
lims = {'time_evo': ('0','100'),
        'volume': ('0','100'),
        'pitch':('0','100')}

# set up source
sources = Objects(maps.keys())
sources.fromdict(maps)
sources.apply_mapping_functions(map_lims=lims)

soni = Sonification(score, sources, generator, system)
soni.render()
dobj = soni.notebook_display()
%matplotlib inline


## 4.2 Using the IFFT "Spectraliser" (fast)

Now lets focus on a single line, and the red and blue shifted components around it, such that the relative shifts are directly audible. We can construct a regularly-spaced spectrum, and display it

In [ ]:
# Pick a line... ✏️
idx = 0
l = lines[idx]

print(f"{l} components")
print(sdata[l]['mdopb'][0], sdata[l]['stddopb'][0])
print(sdata[l]['mdopr'][0], sdata[l]['stddopr'][0])

# many wavelength points as this technique is fast....
wls = np.linspace(175, -175, 100000)

# construct spectrum
spec = gaussian(wls, sdata[l]['mdopb'][-1], sdata[l]['stddopb'][-1])
spec += gaussian(wls, sdata[l]['mdopr'][-1], sdata[l]['stddopr'][-1])

plt.plot(wls, spec)
plt.xlabel("Doppler Velocity")
plt.ylabel("Intensity")

In the code we are now investigating a ***much*** faster but less flexible approach. This uses the discrete, _Inverse Fast Fourier Transform_ algorithm (IFFT).

This will convert a spectrum, scaled to audible frequencies, and generate a signal directly between a maximum and minimum frequency range.

This algorithm is thousands of times faster than the oscillator array approach _(bearing in mind the large overhead on displaying audio files in `Colab`, vs locally)_, but for now isn't an evolvable parameter... This is coming soon!

For now, we just iterate through and evaluate the spectrum across the red and blue shifted components of a single line, and display each spectrum with it's sonification.

✨Evolving IFFT spectralisation **coming soon...**✨

In [ ]:
# set up spectralizer generator
generator = Spectralizer()

# Lets pick the mapping frequency range for the spectrum... ✏️
generator.modify_preset({'min_freq':200, 'max_freq':1000})

score =  Score(notes, 1)

for i in range(sdata[l]['mdopb'].size)[::999]:
    
    spec = gaussian(wls, sdata[l]['mdopb'][i], sdata[l]['stddopb'][i])
    spec += gaussian(wls, sdata[l]['mdopr'][i], sdata[l]['stddopr'][i])
    
    # set up spectrum and choose some envelope parameters for fade-in and fade-out
    maps = {'spectrum': [spec], 'pitch':[1],
            'volume_envelope/D':[0.5], 
            'volume_envelope/S':[0.], 
            'volume_envelope/A':[0.01]}

    # again, use maximal range for the mapped parameters
    lims = {'spectrum': ('0','100')}

    # set up source
    sources = Events(maps.keys())
    sources.fromdict(maps)
    sources.apply_mapping_functions(map_lims=lims)

    # render and play sonification!
    soni = Sonification(score, sources, generator, system)
    soni.render()
    
    plt.plot(wls,spec)
    plt.xlabel("Doppler Velocity")
    plt.ylabel("Intensity")
    plt.show()
    
    dobj = soni.notebook_display()
    
%matplotlib inline

# 6. Sandbox!